<a href="https://colab.research.google.com/github/galenzo17/AI-personal-test/blob/main/Grafo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Código completo para entrenar un modelo y construir un grafo de conocimiento basado en un repositorio de GitHub

# Primero, instalamos los paquetes necesarios
!pip install gitpython networkx matplotlib tqdm

# Importamos las librerías necesarias
import os
import git
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm
import ast
import pickle

# Definimos la URL del repositorio de GitHub (ejemplo: reemplaza con el repositorio que deseas)
GITHUB_REPO_URL = "https://github.com/usuario/repositorio.git"
# Directorio donde se clonará el repositorio
CLONE_DIR = "/content/repositorio_clonado"
# Archivo para almacenar el grafo de conocimiento
GRAPH_PATH = "/content/grafo_conocimiento.pkl"

# Función para clonar el repositorio si no está clonado
def clonar_repositorio(url, path):
    if not os.path.exists(path):
        print(f"Clonando el repositorio desde {url}...")
        git.Repo.clone_from(url, path)
        print("Repositorio clonado exitosamente.")
    else:
        print("El repositorio ya está clonado.")

# Función para cargar el grafo existente o crear uno nuevo
def cargar_grafo(path):
    if os.path.exists(path):
        print("Cargando el grafo de conocimiento existente...")
        with open(path, 'rb') as f:
            G = pickle.load(f)
        print("Grafo cargado.")
    else:
        print("Creando un nuevo grafo de conocimiento.")
        G = nx.DiGraph()
    return G

# Función para guardar el grafo de conocimiento
def guardar_grafo(G, path):
    with open(path, 'wb') as f:
        pickle.dump(G, f)
    print("Grafo de conocimiento guardado.")

# Función para extraer información de archivos Python
def procesar_archivo_python(filepath, G):
    with open(filepath, 'r', encoding='utf-8') as file:
        try:
            tree = ast.parse(file.read(), filename=filepath)
        except Exception as e:
            print(f"Error al parsear {filepath}: {e}")
            return

    for node in ast.walk(tree):
        if isinstance(node, ast.ClassDef):
            clase = f"Clase: {node.name}"
            G.add_node(clase, type='Clase', archivo=filepath)
            # Relaciones con funciones dentro de la clase
            for body_item in node.body:
                if isinstance(body_item, ast.FunctionDef):
                    funcion = f"Función: {body_item.name}"
                    G.add_node(funcion, type='Función', archivo=filepath)
                    G.add_edge(clase, funcion, relationship='contiene')
        elif isinstance(node, ast.FunctionDef):
            funcion = f"Función: {node.name}"
            G.add_node(funcion, type='Función', archivo=filepath)
        elif isinstance(node, ast.Import):
            for alias in node.names:
                mod = alias.name
                G.add_node(mod, type='Módulo')
                G.add_edge(os.path.basename(filepath), mod, relationship='importa')
        elif isinstance(node, ast.ImportFrom):
            mod = node.module
            G.add_node(mod, type='Módulo')
            G.add_edge(os.path.basename(filepath), mod, relationship='importa')

# Función principal para construir el grafo de conocimiento
def construir_grafo(repo_path, graph_path):
    G = cargar_grafo(graph_path)
    # Recorremos todos los archivos Python en el repositorio
    for root, dirs, files in os.walk(repo_path):
        for file in tqdm(files, desc="Procesando archivos"):
            if file.endswith(".py"):
                filepath = os.path.join(root, file)
                G.add_node(os.path.basename(filepath), type='Archivo', ruta=filepath)
                procesar_archivo_python(filepath, G)
    guardar_grafo(G, graph_path)
    return G

# Función para visualizar el grafo
def visualizar_grafo(G, max_nodos=1000):
    plt.figure(figsize=(15, 15))
    pos = nx.spring_layout(G, k=0.15, iterations=20)
    # Limitar el número de nodos para una visualización más clara
    if len(G.nodes) > max_nodos:
        sub_nodes = list(G.nodes)[:max_nodos]
        H = G.subgraph(sub_nodes)
    else:
        H = G
    # Colores por tipo de nodo
    color_map = {'Archivo': 'lightblue', 'Clase': 'lightgreen', 'Función': 'lightcoral', 'Módulo': 'gold'}
    colores = [color_map.get(data['type'], 'gray') for _, data in H.nodes(data=True)]
    nx.draw_networkx_nodes(H, pos, node_size=50, node_color=colores, alpha=0.7)
    nx.draw_networkx_edges(H, pos, alpha=0.5)
    plt.title("Grafo de Conocimiento del Repositorio")
    plt.axis('off')
    plt.show()

# Ejecutamos las funciones definidas
clonar_repositorio(GITHUB_REPO_URL, CLONE_DIR)
grafo = construir_grafo(CLONE_DIR, GRAPH_PATH)
visualizar_grafo(grafo)

Clonando el repositorio desde https://github.com/usuario/repositorio.git...


GitCommandError: Cmd('git') failed due to: exit code(128)
  cmdline: git clone -v -- https://github.com/usuario/repositorio.git /content/repositorio_clonado
  stderr: 'Cloning into '/content/repositorio_clonado'...
fatal: could not read Username for 'https://github.com': No such device or address
'